In [1]:
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
#from glmnet import glmnet; from glmnetPlot import glmnetPlot 
#from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
#from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
#from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict
import glob
import csv
import numpy as np
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
files = glob.glob("../../final_features/*")
print(len(files))

102


In [4]:
print(files)

['../../final_features/TCGA-25-1878.csv', '../../final_features/TCGA-23-2643.csv', '../../final_features/TCGA-23-1809.csv', '../../final_features/TCGA-25-1313.csv', '../../final_features/TCGA-23-1028.csv', '../../final_features/TCGA-23-2079.csv', '../../final_features/TCGA-23-2078.csv', '../../final_features/TCGA-23-1029.csv', '../../final_features/TCGA-25-1312.csv', '../../final_features/TCGA-5X-AA5U.csv', '../../final_features/TCGA-25-2396.csv', '../../final_features/TCGA-23-2084.csv', '../../final_features/TCGA-23-2641.csv', '../../final_features/TCGA-25-2397.csv', '../../final_features/TCGA-25-2393.csv', '../../final_features/TCGA-23-2645.csv', '../../final_features/TCGA-25-1329.csv', '../../final_features/TCGA-25-1315.csv', '../../final_features/TCGA-23-2081.csv', '../../final_features/TCGA-25-1314.csv', '../../final_features/TCGA-25-1328.csv', '../../final_features/TCGA-25-2392.csv', '../../final_features/TCGA-25-2409.csv', '../../final_features/TCGA-25-1316.csv', '../../final_fe

In [5]:
features = []
for file in files:
    flag = -1
    file_features = []
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                for index in range(0, len(array)):
                    file_features.append(float(array[index]))
    features.append(file_features)
print(len(features))

102


In [6]:
train_features = []
val_features = []
for index in range(0, 51):
    current_features = []
    current_features.append(index+1)
    for index1 in range(0, len(features[index])):
        current_features.append(features[index][index1])
    train_features.append(current_features)

for index in range(51, len(features)):
    current_features = []
    current_features.append(index+1-51)
    for index1 in range(0, len(features[index])):
        current_features.append(features[index][index1])
    val_features.append(current_features)

In [7]:
print(len(train_features))
print(len(val_features))

51
51


In [8]:
print(len(train_features[0]))
print(len(val_features[0]))

893
893


In [11]:
rows = []
rows.append("")
for index in range(1, 893):
    rows.append("F_" + str(index))
print(len(rows))

893


In [12]:
final_train_features = []
final_train_features.append(rows)
for index in range(0, len(train_features)):
    final_train_features.append(train_features[index])
print(len(final_train_features))

52


In [13]:
final_val_features = []
final_val_features.append(rows)
for index in range(0, len(val_features)):
    final_val_features.append(val_features[index])
print(len(final_val_features))

52


In [14]:
# write csv file
with open("train_features.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_train_features)):
        spamwriter.writerow(final_train_features[index])

In [15]:
# write csv file
with open("val_features.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_val_features)):
        spamwriter.writerow(final_val_features[index])

In [16]:
flag = -1
new_filenames = []
survival_info = []
with open("../../Ov_TCGA_data.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        file_survival_info = []
        if flag == -1:
            flag = 1
        else:
            array = row
            new_filenames.append(array[1])
            if array[10] == "--":
                if array[len(array)-1] == "--":
                    array[len(array)-1] = 50
                file_survival_info.append(float(array[len(array)-1]))
                file_survival_info.append(0)
            else:
                file_survival_info.append(float(array[10]))
                file_survival_info.append(1)
            survival_info.append(file_survival_info)

In [17]:
y = []
for file in files:
    file = file.split("/")[-1][:-4]
    count = 0
    flag = -1
    for filename in new_filenames:
        if file == filename:
            y.append(survival_info[count])
            flag = 1
        count += 1

In [18]:
print(y)

[[2587.0, 1], [143.0, 0], [16.0, 0], [820.0, 1], [1503.0, 0], [2788.0, 0], [2661.0, 0], [268.0, 0], [31.0, 1], [361.0, 0], [92.0, 1], [1516.0, 1], [84.0, 0], [365.0, 1], [1157.0, 1], [137.0, 0], [457.0, 1], [1583.0, 1], [2342.0, 1], [1004.0, 1], [2009.0, 1], [31.0, 1], [821.0, 1], [1279.0, 1], [61.0, 1], [943.0, 1], [135.0, 0], [1492.0, 1], [627.0, 1], [1013.0, 1], [731.0, 1], [396.0, 1], [346.0, 1], [592.0, 1], [45.0, 1], [722.0, 0], [1240.0, 0], [2089.0, 1], [317.0, 1], [1207.0, 0], [98.0, 0], [53.0, 1], [1658.0, 0], [576.0, 0], [679.0, 0], [9.0, 1], [949.0, 1], [1447.0, 1], [1583.0, 1], [1189.0, 1], [882.0, 0], [763.0, 0], [1018.0, 1], [1091.0, 1], [1562.0, 1], [194.0, 0], [130.0, 0], [565.0, 1], [394.0, 1], [1278.0, 0], [1891.0, 1], [2616.0, 0], [304.0, 1], [761.0, 0], [3953.0, 0], [1799.0, 1], [880.0, 1], [518.0, 1], [1162.0, 1], [840.0, 1], [9.0, 1], [760.0, 1], [1249.0, 1], [1446.0, 1], [883.0, 1], [455.0, 1], [116.0, 0], [976.0, 1], [1977.0, 1], [1511.0, 1], [759.0, 1], [1233.0

In [19]:
train_labels = []
val_labels = []
for index in range(0, 51):
    current_labels = []
    current_labels.append(index+1)
    for index1 in range(0, len(y[index])):
        current_labels.append(y[index][index1])
    train_labels.append(current_labels)
    
for index in range(51, len(y)):
    current_labels = []
    current_labels.append(index+1-51)
    for index1 in range(0, len(y[index])):
        current_labels.append(y[index][index1])
    val_labels.append(current_labels)
print(len(train_labels))
print(len(val_labels))

51
51


In [20]:
rows = []
rows.append("")
rows.append("Survival (in days)")
rows.append("Status")

In [21]:
final_train_labels = []
final_train_labels.append(rows)
for index in range(0, len(train_labels)):
    final_train_labels.append(train_labels[index])
print(len(final_train_labels))

52


In [22]:
final_val_labels = []
final_val_labels.append(rows)
for index in range(0, len(val_labels)):
    final_val_labels.append(val_labels[index])
print(len(final_val_labels))

52


In [23]:
# write csv file
with open("train_labels.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_train_labels)):
        spamwriter.writerow(final_train_labels[index])

In [24]:
# write csv file
with open("val_labels.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_val_labels)):
        spamwriter.writerow(final_val_labels[index])